In [1]:
import es_distributed.tf_util as U
import tensorflow as tf

from es_distributed.policies import catcher, CatchPolicy
from es_distributed.es import *
from es_distributed import policies


exp = {
  "config": {
    "calc_obstat_prob": 0.0,
    "episodes_per_batch": 10000,
    "eval_prob": 0.03,
    "l2coeff": 0.005,
    "noise_stdev": 0.02,
    "snapshot_freq": 5,
    "timesteps_per_batch": 100000,
    "return_proc_mode": "centered_rank",
    "episode_cutoff_mode": "env_default"
  },
  "env_id": "catcher",
  "exp_prefix": "humanoid",
  "optimizer": {
    "args": {
      "stepsize": 0.01
    },
    "type": "adam"
  },
  "policy": {
    "args": {
      "connection_type": "ff",
      "hidden_dims": [
        100,
        100
      ],
      "nonlin_type": "tanh"
    },
    "type": "CatchPolicy"
  }
}


config, env, sess, policy = setup(exp, single_threaded=False)
policy.initialize_from('/home/aignatov/snapshot.h5')
policy.rollout(env)

Instructions for updating:
Please use tf.global_variables instead.
Instructions for updating:
Use `tf.variables_initializer` instead.


(array([    0.,     0.,     0.,     0.,     0.,     0.,     0.,  1500.], dtype=float32),
 8)

In [2]:
import json
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import sgd


nonlin_type = exp['policy']["args"]["nonlin_type"]
ob_space = catcher().observation_space
num_actions = 3

fill_dict = {}
model = Sequential()
for ilayer, hd in enumerate(exp['policy']["args"]['hidden_dims']):
    my_vars = [var for var in tf.trainable_variables() if 'l{}'.format(ilayer) in var.name]
    if ilayer == 0:
        cur_l = Dense(hd, input_shape=ob_space.shape, activation=nonlin_type)
    else:
        cur_l = Dense(hd, activation=nonlin_type)
    model.add(cur_l)
    fill_dict[cur_l] = U.eval(my_vars)

    
    
    
my_vars = [var for var in tf.trainable_variables() if 'out' in var.name]
cur_l = Dense(num_actions)
fill_dict[cur_l] = U.eval(my_vars)

model.add(cur_l)

model.compile(sgd(lr=.2), "mse")

for my_l in model.layers:
    my_l.set_weights(fill_dict[my_l])

Using TensorFlow backend.


In [3]:
model.layers

In [4]:
tf.trainable_variables()

[<tf.Variable 'CatchPolicy/l0/w:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'CatchPolicy/l0/b:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'CatchPolicy/l1/w:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'CatchPolicy/l1/b:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'CatchPolicy/out/w:0' shape=(100, 3) dtype=float32_ref>,
 <tf.Variable 'CatchPolicy/out/b:0' shape=(3,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'dense_3/kernel:0' shape=(100, 3) dtype=float32_ref>,
 <tf.Variable 'dense_3/bias:0' shape=(3,) dtype=float32_ref>,
 <tf.Variable 'SGD/iterations:0' shape=() dtype=int64_ref>,
 <tf.Variable 'SGD/lr:0' shape=() dtype=float32_ref>,
 <tf.Variable 'SGD/momentum:0' shape=() dtype=float32_ref>,
 <tf.Varia

In [5]:
a = env.reset()

In [3]:
a = [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.]

In [4]:
model.predict(np.array([a]))

array([[ 0.04267366, -0.11012069,  0.01793103]], dtype=float32)

In [5]:
policy.act([a])

array([[ 0.04267366, -0.11012069,  0.01793103]], dtype=float32)

In [9]:
policy.act([a])

array([[ 0.04267366, -0.11012069,  0.01793103]], dtype=float32)

In [9]:
for i in range(100):
    print(policy.rollout(env)[0][-1])

-1000.0
1500.0
1000.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
-1000.0
1500.0
-1000.0
1500.0
1500.0
-1000.0
1000.0
1500.0
-1000.0
1000.0
1500.0
1000.0
1500.0
-1000.0
1500.0
1500.0
1000.0
1500.0
1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
-1000.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
-1000.0
-1000.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1000.0
1500.0
-1000.0
-1000.0
1500.0
1000.0
1500.0
1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
-1000.0
1500.0
1500.0
1500.0
1500.0
-1000.0
1500.0
-1000.0
1500.0
1500.0
1500.0


init = tf.initialize_all_variables()
sess.run(init)

x = tf.constant([[1, 1, 1], [1, 1, 1]])
tf.reduce_sum(x)  # 6
tf.reduce_sum(x, 0)  # [2, 2, 2]
tf.reduce_sum(x, 1)  # [3, 3]
a  # [[3], [3]]
tf.reduce_sum(x, [0, 1])  # 6

nextQ = tf.placeholder(shape=policy.a.shape,dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - policy.a),0)

init_new_vars_op = tf.initialize_variables([v_6, v_7, v_8])
sess.run(init_new_vars_op)

In [5]:
nextQ = tf.placeholder(shape=policy.a.shape,dtype=tf.float32)
#loss = tf.losses.mean_squared_error(nextQ, policy.a)
loss = tf.reduce_sum(tf.square(nextQ - policy.a),0)
trainer = tf.train.RMSPropOptimizer(0.05, momentum=0.95, epsilon=0.01)
#trainer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
#trainer = tf.train.AdamOptimizer(learning_rate=0.01)
updateModel = trainer.minimize(loss)
#U.eval(tf.initialize_variables(list(set(tf.all_variables()) - set(tf.trainable_variables()))))
U.eval(tf.initialize_all_variables())

import json
import numpy as np

class Catch(object):
    def __init__(self, grid_size=10):
        self.grid_size = grid_size
        self.reset()

    def _update_state(self, action):
        """
        Input: action and states
        Ouput: new states and reward
        """
        state = self.state
        if action == 0:  # left
            action = -1
        elif action == 1:  # stay
            action = 0
        else:
            action = 1  # right
        f0, f1, basket = state[0]
        new_basket = min(max(1, basket + action), self.grid_size-1)
        f0 += 1
        out = np.asarray([f0, f1, new_basket])
        out = out[np.newaxis]

        assert len(out.shape) == 2
        self.state = out

    def _draw_state(self):
        im_size = (self.grid_size,)*2
        state = self.state[0]
        canvas = np.zeros(im_size)
        canvas[state[0], state[1]] = 1  # draw fruit
        canvas[-1, state[2]-1:state[2] + 2] = 1  # draw basket
        return canvas

    def _get_reward(self):
        fruit_row, fruit_col, basket = self.state[0]
        if fruit_row == self.grid_size-1:
            if abs(fruit_col - basket) <= 1:
                return 1000
            else:
                return -1000
        else:
            return 0

    def _is_over(self):
        if self.state[0, 0] == self.grid_size-1:
            return True
        else:
            return False

    def observe(self):
        canvas = self._draw_state()
        return canvas.reshape((1, -1))

    def act(self, action):
        self._update_state(action)
        reward = self._get_reward()
        game_over = self._is_over()
        return self.observe(), reward, game_over

    def reset(self):
        n = np.random.randint(0, self.grid_size-1, size=1)
        m = np.random.randint(1, self.grid_size-2, size=1)
        self.state = np.asarray([0, n, m])[np.newaxis]


class ExperienceReplay(object):
    def __init__(self, max_memory=100, discount=.9):
        self.max_memory = max_memory
        self.memory = list()
        self.discount = discount

    def remember(self, states, game_over):
        # memory[i] = [[state_t, action_t, reward_t, state_t+1], game_over?]
        self.memory.append([states, game_over])
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def get_batch(self, policy, batch_size=10):
        len_memory = len(self.memory)
        num_actions = int(policy.a.shape[-1])
        env_dim = self.memory[0][0][0].shape[1]
        inputs = np.zeros((min(len_memory, batch_size), env_dim))
        targets = np.zeros((inputs.shape[0], num_actions))
        for i, idx in enumerate(np.random.randint(0, len_memory,
                                                  size=inputs.shape[0])):
            state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
            game_over = self.memory[idx][1]

            inputs[i:i+1] = state_t
            # There should be no target values for actions not taken.
            # Thou shalt not correct actions not taken #deep
            targets[i] = policy.act(state_t)[0]
            Q_sa = np.max(policy.act(state_tp1)[0])
            if game_over:  # if game_over is True
                targets[i, action_t] = reward_t
            else:
                # reward_t + gamma * max_a' Q(s', a')
                targets[i, action_t] = reward_t + self.discount * Q_sa
        return inputs, targets


epsilon = .1  # exploration
num_actions = 3  # [move_left, stay, move_right]
epoch = 100000
max_memory = 500
hidden_size = 100
batch_size = 100
grid_size = 10
avg_pct = 0
#model = Sequential()
#model.add(Dense(hidden_size, input_shape=(grid_size**2,), activation='relu'))
#model.add(Dense(hidden_size, activation='relu'))
#model.add(Dense(num_actions))
#model.compile(sgd(lr=.2), "mse")

# If you want to continue training from a previous model, just uncomment the line bellow
# model.load_weights("model.h5")

# Define environment/game
env = Catch(grid_size)

# Initialize experience replay object
exp_replay = ExperienceReplay(max_memory=max_memory)

# Train
win_cnt = 0
for e in range(epoch):
    loss = 0.
    env.reset()
    game_over = False
    # get initial input
    input_t = env.observe()

    while not game_over:
        input_tm1 = input_t
        # get next action
        if np.random.rand() <= epsilon:
            action = np.random.randint(0, num_actions, size=1)
        else:
            q = policy.act(input_tm1)
            action = np.argmax(q[0])

        # apply action, get rewards and new state
        input_t, reward, game_over = env.act(action)
        if reward > 0:
            win_cnt += 1

        # store experience
        exp_replay.remember([input_tm1, action, reward, input_t], game_over)

    # adapt model
        inputs, targets = exp_replay.get_batch(policy, batch_size=batch_size)
        U.eval(updateModel, {policy.o:inputs,nextQ:targets})
    is_won = int(reward  > 0)
    avg_pct = .1*is_won + .9*avg_pct

    print("Epoch {:03d}/999 | Win count {}| {}".format(e, win_cnt, avg_pct))



Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 000/999 | Win count 0| 0.0
Epoch 001/999 | Win count 1| 0.1
Epoch 002/999 | Win count 1| 0.09000000000000001
Epoch 003/999 | Win count 2| 0.18100000000000002
Epoch 004/999 | Win count 2| 0.16290000000000002
Epoch 005/999 | Win count 2| 0.14661000000000002
Epoch 006/999 | Win count 2| 0.131949
Epoch 007/999 | Win count 2| 0.11875410000000002
Epoch 008/999 | Win count 3| 0.20687869000000003
Epoch 009/999 | Win count 3| 0.18619082100000003
Epoch 010/999 | Win count 4| 0.2675717389000001
Epoch 011/999 | Win count 5| 0.34081456501000007
Epoch 012/999 | Win count 6| 0.40673310850900013
Epoch 013/999 | Win count 6| 0.36605979765810015
Epoch 014/999 | Win count 6| 0.32945381789229017
Epoch 015/999 | Win count 7| 0.3965084361030612
Epoch 016/999 | Win count 8| 0.45685759249275504
Epoch 017/999 | Win count 9| 0.5111718332434796
Epoch 018/999 | Win count 9| 0.46005464991913164
Epoch 019/999 | Win count 9| 0.4140491849

KeyboardInterrupt: 

In [3]:
policy.a

<tf.Tensor 'CatchPolicy/add_2:0' shape=(?, 3) dtype=float32>

In [5]:
env = catcher()
for i in range(100):
    print(policy.rollout(env)[0][-1])

-1000.0
1000.0
1000.0
-1000.0
1000.0
1500.0
1000.0
1500.0
-1000.0
-1000.0
1000.0
1000.0
-1000.0
1000.0
-1000.0
-1000.0
-1000.0
-1000.0
1500.0
1000.0
-1000.0
-1000.0
1500.0
-1000.0
-1000.0
-1000.0
-1000.0
1000.0
-1000.0
-1000.0
-1000.0
1500.0
-1000.0
-1000.0
1000.0
-1000.0
1000.0
-1000.0
1000.0
-1000.0
1000.0
1000.0
1500.0
1000.0
-1000.0
-1000.0
1000.0
-1000.0
-1000.0
-1000.0
1500.0
-1000.0
1500.0
1000.0
1500.0
1500.0
-1000.0
-1000.0
-1000.0
-1000.0
1000.0
1000.0
-1000.0
-1000.0
1500.0
-1000.0
1000.0
1500.0
-1000.0
1000.0
-1000.0
1000.0
1000.0
-1000.0
1000.0
-1000.0
1000.0
-1000.0
-1000.0
1000.0
-1000.0
-1000.0
-1000.0
-1000.0
1000.0
-1000.0
1500.0
1500.0
-1000.0
-1000.0
1000.0
1500.0
-1000.0
-1000.0
1000.0
1000.0
-1000.0
-1000.0
1000.0
-1000.0


# standard Qlearn

In [ ]:
import json
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import sgd


class Catch(object):
    def __init__(self, grid_size=10):
        self.grid_size = grid_size
        self.reset()

    def _update_state(self, action):
        """
        Input: action and states
        Ouput: new states and reward
        """
        state = self.state
        if action == 0:  # left
            action = -1
        elif action == 1:  # stay
            action = 0
        else:
            action = 1  # right
        f0, f1, basket = state[0]
        new_basket = min(max(1, basket + action), self.grid_size-1)
        f0 += 1
        out = np.asarray([f0, f1, new_basket])
        out = out[np.newaxis]

        assert len(out.shape) == 2
        self.state = out

    def _draw_state(self):
        im_size = (self.grid_size,)*2
        state = self.state[0]
        canvas = np.zeros(im_size)
        canvas[state[0], state[1]] = 1  # draw fruit
        canvas[-1, state[2]-1:state[2] + 2] = 1  # draw basket
        return canvas

    def _get_reward(self):
        fruit_row, fruit_col, basket = self.state[0]
        if fruit_row == self.grid_size-1:
            if abs(fruit_col - basket) <= 1:
                return 1
            else:
                return -1
        else:
            return 0

    def _is_over(self):
        if self.state[0, 0] == self.grid_size-1:
            return True
        else:
            return False

    def observe(self):
        canvas = self._draw_state()
        return canvas.reshape((1, -1))

    def act(self, action):
        self._update_state(action)
        reward = self._get_reward()
        game_over = self._is_over()
        return self.observe(), reward, game_over

    def reset(self):
        n = np.random.randint(0, self.grid_size-1, size=1)
        m = np.random.randint(1, self.grid_size-2, size=1)
        self.state = np.asarray([0, n, m])[np.newaxis]


class ExperienceReplay(object):
    def __init__(self, max_memory=100, discount=.9):
        self.max_memory = max_memory
        self.memory = list()
        self.discount = discount

    def remember(self, states, game_over):
        # memory[i] = [[state_t, action_t, reward_t, state_t+1], game_over?]
        self.memory.append([states, game_over])
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def get_batch(self, model, batch_size=10):
        len_memory = len(self.memory)
        num_actions = model.output_shape[-1]
        env_dim = self.memory[0][0][0].shape[1]
        inputs = np.zeros((min(len_memory, batch_size), env_dim))
        targets = np.zeros((inputs.shape[0], num_actions))
        for i, idx in enumerate(np.random.randint(0, len_memory,
                                                  size=inputs.shape[0])):
            state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
            game_over = self.memory[idx][1]

            inputs[i:i+1] = state_t
            # There should be no target values for actions not taken.
            # Thou shalt not correct actions not taken #deep
            targets[i] = model.predict(state_t)[0]
            Q_sa = np.max(model.predict(state_tp1)[0])
            if game_over:  # if game_over is True
                targets[i, action_t] = reward_t
            else:
                # reward_t + gamma * max_a' Q(s', a')
                targets[i, action_t] = reward_t + self.discount * Q_sa
        return inputs, targets


if __name__ == "__main__":
    # parameters
    epsilon = .1  # exploration
    num_actions = 3  # [move_left, stay, move_right]
    epoch = 10000
    max_memory = 500
    hidden_size = 100
    batch_size = 50
    grid_size = 10
    avg_pct = 0

    #model = Sequential()
    #model.add(Dense(hidden_size, input_shape=(grid_size**2,), activation='relu'))
    #model.add(Dense(hidden_size, activation='relu'))
    #model.add(Dense(num_actions))
    #model.compile(sgd(lr=.2), "mse")
    
    
    

    nonlin_type = exp['policy']["args"]["nonlin_type"]
    ob_space = catcher().observation_space
    num_actions = 3

    fill_dict = {}
    model = Sequential()
    for ilayer, hd in enumerate(exp['policy']["args"]['hidden_dims']):
        my_vars = [var for var in tf.trainable_variables() if 'l{}'.format(ilayer) in var.name]
        if ilayer == 0:
            cur_l = Dense(hd, input_shape=ob_space.shape, activation=nonlin_type)
        else:
            cur_l = Dense(hd, activation=nonlin_type)
        model.add(cur_l)
        fill_dict[cur_l] = U.eval(my_vars)




    my_vars = [var for var in tf.trainable_variables() if 'out' in var.name]
    cur_l = Dense(num_actions)
    fill_dict[cur_l] = U.eval(my_vars)

    model.add(cur_l)

    model.compile(sgd(lr=.2), "mse")

    for my_l in model.layers:
        my_l.set_weights(fill_dict[my_l])

    # If you want to continue training from a previous model, just uncomment the line bellow
    # model.load_weights("model.h5")

    # Define environment/game
    env = Catch(grid_size)

    # Initialize experience replay object
    exp_replay = ExperienceReplay(max_memory=max_memory)

    # Train
    win_cnt = 0
    for e in range(epoch):
        loss = 0.
        env.reset()
        game_over = False
        # get initial input
        input_t = env.observe()

        while not game_over:
            input_tm1 = input_t
            # get next action
            if np.random.rand() <= epsilon:
                action = np.random.randint(0, num_actions, size=1)
            else:
                q = model.predict(input_tm1)
                action = np.argmax(q[0])

            # apply action, get rewards and new state
            input_t, reward, game_over = env.act(action)
            if reward == 1:
                win_cnt += 1

            # store experience
            exp_replay.remember([input_tm1, action, reward, input_t], game_over)

            # adapt model
            inputs, targets = exp_replay.get_batch(model, batch_size=batch_size)

            #loss += model.train_on_batch(inputs, targets)
        
        is_won = int(reward == 1)
        avg_pct = .1*is_won + .9*avg_pct
        print("Epoch {:03d}/999 | Loss {:.4f} | Win count {}| pct {}".format(e, loss, win_cnt, avg_pct))



Using TensorFlow backend.


Epoch 000/999 | Loss 0.0000 | Win count 1| pct 0.1
Epoch 001/999 | Loss 0.0000 | Win count 2| pct 0.19
Epoch 002/999 | Loss 0.0000 | Win count 2| pct 0.171
Epoch 003/999 | Loss 0.0000 | Win count 3| pct 0.2539
Epoch 004/999 | Loss 0.0000 | Win count 4| pct 0.32851
Epoch 005/999 | Loss 0.0000 | Win count 5| pct 0.395659
Epoch 006/999 | Loss 0.0000 | Win count 6| pct 0.45609310000000003
Epoch 007/999 | Loss 0.0000 | Win count 6| pct 0.41048379
Epoch 008/999 | Loss 0.0000 | Win count 7| pct 0.4694354110000001
Epoch 009/999 | Loss 0.0000 | Win count 8| pct 0.5224918699000001
Epoch 010/999 | Loss 0.0000 | Win count 9| pct 0.5702426829100001
Epoch 011/999 | Loss 0.0000 | Win count 10| pct 0.6132184146190001
Epoch 012/999 | Loss 0.0000 | Win count 11| pct 0.6518965731571
Epoch 013/999 | Loss 0.0000 | Win count 11| pct 0.58670691584139
Epoch 014/999 | Loss 0.0000 | Win count 11| pct 0.528036224257251
Epoch 015/999 | Loss 0.0000 | Win count 12| pct 0.5752326018315259
Epoch 016/999 | Loss 0.0000

Epoch 125/999 | Loss 0.0000 | Win count 98| pct 0.7016828514613637
Epoch 126/999 | Loss 0.0000 | Win count 99| pct 0.7315145663152273
Epoch 127/999 | Loss 0.0000 | Win count 100| pct 0.7583631096837046
Epoch 128/999 | Loss 0.0000 | Win count 101| pct 0.7825267987153341
Epoch 129/999 | Loss 0.0000 | Win count 102| pct 0.8042741188438007
Epoch 130/999 | Loss 0.0000 | Win count 102| pct 0.7238467069594207
Epoch 131/999 | Loss 0.0000 | Win count 103| pct 0.7514620362634786
Epoch 132/999 | Loss 0.0000 | Win count 104| pct 0.7763158326371308
Epoch 133/999 | Loss 0.0000 | Win count 104| pct 0.6986842493734177
Epoch 134/999 | Loss 0.0000 | Win count 104| pct 0.6288158244360759
Epoch 135/999 | Loss 0.0000 | Win count 105| pct 0.6659342419924683
Epoch 136/999 | Loss 0.0000 | Win count 105| pct 0.5993408177932215
Epoch 137/999 | Loss 0.0000 | Win count 105| pct 0.5394067360138993
Epoch 138/999 | Loss 0.0000 | Win count 106| pct 0.5854660624125094
Epoch 139/999 | Loss 0.0000 | Win count 106| pct 0

Epoch 246/999 | Loss 0.0000 | Win count 182| pct 0.790612303057616
Epoch 247/999 | Loss 0.0000 | Win count 183| pct 0.8115510727518543
Epoch 248/999 | Loss 0.0000 | Win count 184| pct 0.8303959654766689
Epoch 249/999 | Loss 0.0000 | Win count 185| pct 0.847356368929002
Epoch 250/999 | Loss 0.0000 | Win count 185| pct 0.7626207320361018
Epoch 251/999 | Loss 0.0000 | Win count 186| pct 0.7863586588324917
Epoch 252/999 | Loss 0.0000 | Win count 187| pct 0.8077227929492425
Epoch 253/999 | Loss 0.0000 | Win count 188| pct 0.8269505136543183
Epoch 254/999 | Loss 0.0000 | Win count 189| pct 0.8442554622888865
Epoch 255/999 | Loss 0.0000 | Win count 189| pct 0.7598299160599978
Epoch 256/999 | Loss 0.0000 | Win count 190| pct 0.7838469244539981
Epoch 257/999 | Loss 0.0000 | Win count 191| pct 0.8054622320085982
Epoch 258/999 | Loss 0.0000 | Win count 192| pct 0.8249160088077384
Epoch 259/999 | Loss 0.0000 | Win count 193| pct 0.8424244079269646
Epoch 260/999 | Loss 0.0000 | Win count 194| pct 0

Epoch 367/999 | Loss 0.0000 | Win count 269| pct 0.7486827097598133
Epoch 368/999 | Loss 0.0000 | Win count 269| pct 0.673814438783832
Epoch 369/999 | Loss 0.0000 | Win count 270| pct 0.7064329949054488
Epoch 370/999 | Loss 0.0000 | Win count 271| pct 0.7357896954149039
Epoch 371/999 | Loss 0.0000 | Win count 272| pct 0.7622107258734135
Epoch 372/999 | Loss 0.0000 | Win count 273| pct 0.7859896532860722
Epoch 373/999 | Loss 0.0000 | Win count 273| pct 0.707390687957465
Epoch 374/999 | Loss 0.0000 | Win count 274| pct 0.7366516191617185
Epoch 375/999 | Loss 0.0000 | Win count 275| pct 0.7629864572455467
Epoch 376/999 | Loss 0.0000 | Win count 276| pct 0.786687811520992
Epoch 377/999 | Loss 0.0000 | Win count 277| pct 0.8080190303688928
Epoch 378/999 | Loss 0.0000 | Win count 277| pct 0.7272171273320035
Epoch 379/999 | Loss 0.0000 | Win count 277| pct 0.6544954145988031
Epoch 380/999 | Loss 0.0000 | Win count 277| pct 0.5890458731389229
Epoch 381/999 | Loss 0.0000 | Win count 278| pct 0.

...

Epoch 851/999 | Loss 0.0000 | Win count 629| pct 0.43154984996142165
Epoch 852/999 | Loss 0.0000 | Win count 629| pct 0.3883948649652795
Epoch 853/999 | Loss 0.0000 | Win count 630| pct 0.4495553784687515
Epoch 854/999 | Loss 0.0000 | Win count 631| pct 0.5045998406218763
Epoch 855/999 | Loss 0.0000 | Win count 632| pct 0.5541398565596888
Epoch 856/999 | Loss 0.0000 | Win count 633| pct 0.5987258709037199
Epoch 857/999 | Loss 0.0000 | Win count 634| pct 0.6388532838133478
Epoch 858/999 | Loss 0.0000 | Win count 635| pct 0.674967955432013
Epoch 859/999 | Loss 0.0000 | Win count 636| pct 0.7074711598888117
Epoch 860/999 | Loss 0.0000 | Win count 637| pct 0.7367240438999305
Epoch 861/999 | Loss 0.0000 | Win count 638| pct 0.7630516395099375
Epoch 862/999 | Loss 0.0000 | Win count 639| pct 0.7867464755589437
Epoch 863/999 | Loss 0.0000 | Win count 640| pct 0.8080718280030493
Epoch 864/999 | Loss 0.0000 | Win count 641| pct 0.8272646452027443
Epoch 865/999 | Loss 0.0000 | Win count 642| pct

Epoch 972/999 | Loss 0.0000 | Win count 721| pct 0.8644556715818152
Epoch 973/999 | Loss 0.0000 | Win count 721| pct 0.7780101044236337
Epoch 974/999 | Loss 0.0000 | Win count 722| pct 0.8002090939812704
Epoch 975/999 | Loss 0.0000 | Win count 722| pct 0.7201881845831434
Epoch 976/999 | Loss 0.0000 | Win count 723| pct 0.748169366124829
Epoch 977/999 | Loss 0.0000 | Win count 723| pct 0.6733524295123461
Epoch 978/999 | Loss 0.0000 | Win count 724| pct 0.7060171865611115
Epoch 979/999 | Loss 0.0000 | Win count 725| pct 0.7354154679050003
Epoch 980/999 | Loss 0.0000 | Win count 726| pct 0.7618739211145003
Epoch 981/999 | Loss 0.0000 | Win count 727| pct 0.7856865290030502
Epoch 982/999 | Loss 0.0000 | Win count 727| pct 0.7071178761027452
Epoch 983/999 | Loss 0.0000 | Win count 727| pct 0.6364060884924707
Epoch 984/999 | Loss 0.0000 | Win count 728| pct 0.6727654796432236
Epoch 985/999 | Loss 0.0000 | Win count 729| pct 0.7054889316789013
Epoch 986/999 | Loss 0.0000 | Win count 730| pct 

Epoch 1092/999 | Loss 0.0000 | Win count 805| pct 0.650604863388029
Epoch 1093/999 | Loss 0.0000 | Win count 806| pct 0.6855443770492261
Epoch 1094/999 | Loss 0.0000 | Win count 806| pct 0.6169899393443035
Epoch 1095/999 | Loss 0.0000 | Win count 807| pct 0.6552909454098731
Epoch 1096/999 | Loss 0.0000 | Win count 808| pct 0.6897618508688858
Epoch 1097/999 | Loss 0.0000 | Win count 809| pct 0.7207856657819972
Epoch 1098/999 | Loss 0.0000 | Win count 810| pct 0.7487070992037975
Epoch 1099/999 | Loss 0.0000 | Win count 811| pct 0.7738363892834178
Epoch 1100/999 | Loss 0.0000 | Win count 812| pct 0.796452750355076
Epoch 1101/999 | Loss 0.0000 | Win count 813| pct 0.8168074753195684
Epoch 1102/999 | Loss 0.0000 | Win count 814| pct 0.8351267277876115
Epoch 1103/999 | Loss 0.0000 | Win count 815| pct 0.8516140550088503
Epoch 1104/999 | Loss 0.0000 | Win count 816| pct 0.8664526495079653
Epoch 1105/999 | Loss 0.0000 | Win count 816| pct 0.7798073845571688
Epoch 1106/999 | Loss 0.0000 | Win c

Epoch 1211/999 | Loss 0.0000 | Win count 898| pct 0.820825452671409
Epoch 1212/999 | Loss 0.0000 | Win count 899| pct 0.8387429074042682
Epoch 1213/999 | Loss 0.0000 | Win count 900| pct 0.8548686166638413
Epoch 1214/999 | Loss 0.0000 | Win count 901| pct 0.8693817549974572
Epoch 1215/999 | Loss 0.0000 | Win count 902| pct 0.8824435794977115
Epoch 1216/999 | Loss 0.0000 | Win count 903| pct 0.8941992215479403
Epoch 1217/999 | Loss 0.0000 | Win count 904| pct 0.9047792993931463
Epoch 1218/999 | Loss 0.0000 | Win count 905| pct 0.9143013694538317
Epoch 1219/999 | Loss 0.0000 | Win count 906| pct 0.9228712325084485
Epoch 1220/999 | Loss 0.0000 | Win count 906| pct 0.8305841092576036
Epoch 1221/999 | Loss 0.0000 | Win count 907| pct 0.8475256983318432
Epoch 1222/999 | Loss 0.0000 | Win count 908| pct 0.8627731284986588
Epoch 1223/999 | Loss 0.0000 | Win count 909| pct 0.876495815648793
Epoch 1224/999 | Loss 0.0000 | Win count 909| pct 0.7888462340839137
Epoch 1225/999 | Loss 0.0000 | Win c

Epoch 1330/999 | Loss 0.0000 | Win count 983| pct 0.6253821767096456
Epoch 1331/999 | Loss 0.0000 | Win count 984| pct 0.662843959038681
Epoch 1332/999 | Loss 0.0000 | Win count 984| pct 0.596559563134813
Epoch 1333/999 | Loss 0.0000 | Win count 985| pct 0.6369036068213316
Epoch 1334/999 | Loss 0.0000 | Win count 986| pct 0.6732132461391984
Epoch 1335/999 | Loss 0.0000 | Win count 987| pct 0.7058919215252786
Epoch 1336/999 | Loss 0.0000 | Win count 987| pct 0.6353027293727508
Epoch 1337/999 | Loss 0.0000 | Win count 988| pct 0.6717724564354757
Epoch 1338/999 | Loss 0.0000 | Win count 988| pct 0.6045952107919281
Epoch 1339/999 | Loss 0.0000 | Win count 989| pct 0.6441356897127353
Epoch 1340/999 | Loss 0.0000 | Win count 990| pct 0.6797221207414618
Epoch 1341/999 | Loss 0.0000 | Win count 990| pct 0.6117499086673156
Epoch 1342/999 | Loss 0.0000 | Win count 991| pct 0.6505749178005841
Epoch 1343/999 | Loss 0.0000 | Win count 991| pct 0.5855174260205257
Epoch 1344/999 | Loss 0.0000 | Win c

In [9]:
policy.get_trainable_flat().shape

(20503,)

In [50]:
my_l = model.layers[0]

In [51]:
my_l.set_weights(U.eval(tf.trainable_variables()[:2]))

ValueError: You called `set_weights(weights)` on layer "dense_10" with a  weight list of length 2, but the layer was expecting 0 weights. Provided weights: [array([[ 0.04432327,  0.00766535,  0.09602918, .....

In [54]:
ob_space.shape

(100,)

In [38]:
[None] + list(.shape)

[None, 100]

In [17]:
model = Sequential()
    model.add(Dense(hidden_size, input_shape=(grid_size**2,), activation='relu'))
    model.add(Dense(hidden_size, activation='relu'))
    model.add(Dense(num_actions))
    model.compile(sgd(lr=.2), "mse")

(100, 100)

In [7]:
import json
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import sgd


nonlin_type = 'relu'
ob_space = catcher().observation_space
num_actions = 3

fill_dict = {}
model = Sequential()
for ilayer, hd in enumerate(exp['policy']["args"]['hidden_dims']):
    my_vars = [var for var in tf.trainable_variables() if 'l{}'.format(ilayer) in var.name]
    if ilayer == 0:
        cur_l = Dense(hd, input_shape=ob_space.shape, activation=nonlin_type)
    else:
        cur_l = Dense(hd, activation=nonlin_type)
    model.add(cur_l)
    fill_dict[cur_l] = U.eval(my_vars)

    
    
    
my_vars = [var for var in tf.trainable_variables() if 'out' in var.name]
cur_l = Dense(num_actions)
fill_dict[cur_l] = U.eval(my_vars)

model.add(Dense(num_actions))

model.compile(sgd(lr=.2), "mse")

Using TensorFlow backend.


In [62]:
my_vars

[<tf.Variable 'CatchPolicy/out/w:0' shape=(100, 3) dtype=float32_ref>,
 <tf.Variable 'CatchPolicy/out/b:0' shape=(3,) dtype=float32_ref>]

In [59]:
for my_l in model.layers:
    my_l.set_weights(fill_dict[my_l])

In [60]:
fill_dict[model.layers[0]]

[array([[ 0.04432327,  0.00766535,  0.09602918, ...,  0.03341845,
         -0.00035556, -0.02659388],
        [-0.09617916, -0.05765538, -0.02316796, ...,  0.0198922 ,
         -0.07383387,  0.05263451],
        [-0.04672238,  0.02437663, -0.01729172, ..., -0.02043067,
          0.02488345,  0.00276561],
        ..., 
        [-0.03794552,  0.00169713,  0.06681889, ...,  0.14633179,
          0.03909836,  0.04491954],
        [ 0.00972021,  0.00508771, -0.04924688, ...,  0.01314818,
          0.03613656, -0.06983208],
        [-0.00642225, -0.01271942, -0.00900328, ...,  0.02705005,
          0.05465912,  0.07533411]], dtype=float32),
 array([-0.00196438, -0.06098351,  0.04765449, -0.07695587,  0.12493545,
         0.00219061,  0.11877503,  0.08942725,  0.00158901,  0.02048279,
        -0.07194519,  0.05281372,  0.05851415,  0.05235613,  0.0344964 ,
         0.04042572, -0.01406628,  0.01120312, -0.05647016, -0.007001  ,
         0.04963955,  0.0133331 , -0.0746529 ,  0.01219887,  0.06

In [25]:
tf.trainable_variables()

[<tf.Variable 'CatchPolicy/l0/w:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'CatchPolicy/l0/b:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'CatchPolicy/l1/w:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'CatchPolicy/l1/b:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'CatchPolicy/out/w:0' shape=(100, 3) dtype=float32_ref>,
 <tf.Variable 'CatchPolicy/out/b:0' shape=(3,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'dense_3/kernel:0' shape=(100, 3) dtype=float32_ref>,
 <tf.Variable 'dense_3/bias:0' shape=(3,) dtype=float32_ref>,
 <tf.Variable 'SGD/iterations:0' shape=() dtype=int64_ref>,
 <tf.Variable 'SGD/lr:0' shape=() dtype=float32_ref>,
 <tf.Variable 'SGD/momentum:0' shape=() dtype=float32_ref>,
 <tf.Varia

In [21]:
U.eval(tf.trainable_variables()[:2])

[array([[ 0.04432327,  0.00766535,  0.09602918, ...,  0.03341845,
         -0.00035556, -0.02659388],
        [-0.09617916, -0.05765538, -0.02316796, ...,  0.0198922 ,
         -0.07383387,  0.05263451],
        [-0.04672238,  0.02437663, -0.01729172, ..., -0.02043067,
          0.02488345,  0.00276561],
        ..., 
        [-0.03794552,  0.00169713,  0.06681889, ...,  0.14633179,
          0.03909836,  0.04491954],
        [ 0.00972021,  0.00508771, -0.04924688, ...,  0.01314818,
          0.03613656, -0.06983208],
        [-0.00642225, -0.01271942, -0.00900328, ...,  0.02705005,
          0.05465912,  0.07533411]], dtype=float32),
 array([-0.00196438, -0.06098351,  0.04765449, -0.07695587,  0.12493545,
         0.00219061,  0.11877503,  0.08942725,  0.00158901,  0.02048279,
        -0.07194519,  0.05281372,  0.05851415,  0.05235613,  0.0344964 ,
         0.04042572, -0.01406628,  0.01120312, -0.05647016, -0.007001  ,
         0.04963955,  0.0133331 , -0.0746529 ,  0.01219887,  0.06

In [6]:
env = catcher()
policy.act([env.reset()])[0]

array([ 615.98175049,  636.49072266,  616.65515137], dtype=float32)

In [13]:
loss

1.2342154320776899e-07

NameError: name 'inputs' is not defined

In [ ]:
_,W1 = sess.run([updateModel,W],feed_dict={inputs1:inputs,nextQ:targets})